## Part 1: Preprocessing

In [136]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [137]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [138]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [139]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']

# Create X_df using your selected columns
x_df = attrition_df[X_columns]

# Show the data types for X_df
x_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [140]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, random_state=78)

In [141]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['Overtime_Binary'] = X_train['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
X_test['Overtime_Binary'] = X_test['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)

X_train = X_train.drop('OverTime', axis=1)
X_test = X_test.drop('OverTime', axis=1)
X_train['Overtime_Binary'].value_counts()


Overtime_Binary
0    799
1    303
Name: count, dtype: int64

In [142]:
# Create a StandardScaler
standard_scaler = StandardScaler()

# Fit the StandardScaler to the training data
standard_scaler.fit(X_train)

# Scale the training and testing data
X_test_scaled = standard_scaler.transform(X_test)
X_train_scaled = standard_scaler.transform(X_train)




In [143]:
# Create a OneHotEncoder for the Department column
dep_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
dep_encoder.set_output(transform="pandas")
# Fit the encoder to the training data
dep_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
encoded_dept_train = dep_encoder.transform(y_train[['Department']])
encoded_dept_test = dep_encoder.transform(y_test[['Department']])



In [144]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
attr_encoder.set_output(transform="pandas")
# Fit the encoder to the training data
attr_encoder.fit(y_train[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data
encoded_attr_train = attr_encoder.transform(y_train[['Attrition']])
encoded_attr_test = attr_encoder.transform(y_test[['Attrition']])



## Create, Compile, and Train the Model

In [145]:
# Find the number of columns in the X training data
col_num = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(col_num,), name='input_layer')


# Create at least two shared layers
shared_layer1 = layers.Dense(units=64, activation='relu', name='shared_layer_1')(input_layer)
shared_layer2 = layers.Dense(units=128, activation='relu', name='shared_layer_2')(shared_layer1)


In [146]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dep_hidden_layer = layers.Dense(units=32, activation='relu', name='department_hidden_layer')(shared_layer2)

# Create the output layer
dep_branch = layers.Dense(units=3, activation='sigmoid', name='department_output_layer')(dep_hidden_layer)


In [147]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden_layer = layers.Dense(units=32, activation='relu', name='attrition_hidden_layer')(shared_layer2)

# Create the output layer
attr_branch = layers.Dense(units=2, activation='sigmoid', name='attrition_output_layer')(attr_hidden_layer)



In [148]:
# Create the model
model = Model(inputs=input_layer, outputs=[dep_branch, attr_branch])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output_layer': 'categorical_crossentropy', 'attrition_output_layer': 'binary_crossentropy'},
              metrics={'department_output_layer': 'accuracy', 'attrition_output_layer': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer_1      │ (None, 64)        │        704 │ input_layer[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer_2      │ (None, 128)       │      8,320 │ shared_layer_1[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden_… │ (None, 32)        │      4,128 │ shared_layer_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden_l… │ (None, 32)        │      4,128 │ shared_layer_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output_… │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output_l… │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [149]:
# Train the model
model.fit(X_train_scaled, [encoded_dept_train, encoded_attr_train], epochs=100, shuffle=True, verbose=2) # found this version works in python 3.10

Epoch 1/100
35/35 - 1s - 32ms/step - attrition_output_layer_accuracy: 0.8240 - attrition_output_layer_loss: 0.4818 - department_output_layer_accuracy: 0.6642 - department_output_layer_loss: 0.5532 - loss: 1.0344
Epoch 2/100
35/35 - 0s - 1ms/step - attrition_output_layer_accuracy: 0.8394 - attrition_output_layer_loss: 0.4090 - department_output_layer_accuracy: 0.6661 - department_output_layer_loss: 0.4706 - loss: 0.8779
Epoch 3/100
35/35 - 0s - 1ms/step - attrition_output_layer_accuracy: 0.8539 - attrition_output_layer_loss: 0.3841 - department_output_layer_accuracy: 0.6670 - department_output_layer_loss: 0.4667 - loss: 0.8512
Epoch 4/100
35/35 - 0s - 1ms/step - attrition_output_layer_accuracy: 0.8548 - attrition_output_layer_loss: 0.3737 - department_output_layer_accuracy: 0.6670 - department_output_layer_loss: 0.4633 - loss: 0.8333
Epoch 5/100
35/35 - 0s - 1ms/step - attrition_output_layer_accuracy: 0.8612 - attrition_output_layer_loss: 0.3556 - department_output_layer_accuracy: 0.668

In [150]:

# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, [encoded_dept_test, encoded_attr_test], verbose=2) # found this version works in python 3.10


12/12 - 0s - 11ms/step - attrition_output_layer_accuracy: 0.8234 - attrition_output_layer_loss: 1.3623 - department_output_layer_accuracy: 0.5082 - department_output_layer_loss: 1.5365 - loss: 2.9249


In [151]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[3]*100}")
print(f"Attrition Accuracy: {test_results[4]*100}")

Department Accuracy: 82.3369562625885
Attrition Accuracy: 50.815218687057495


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric for this dataset.  The Attrition task likely involves an imbalanced dataset (e.g., fewer employees leave than stay), which can result in misleading accuracy.  The Department task is a multi-class classification problem, and while accuracy is commonly used, other metrics might provide a better view across all classes.
2. For the both Attrition and Department targets sigmoid was used.  Attrition is technically a binary choice, and Department was multi-class, but contained classifications that were not related, so sigmoid seemed a good candidate both.  
3. Use alternative metrics for measuring the model is a good start.  Use precision, recall, and F1-score (weighted F1 score for Department).  Maybe add more features to improve the model.  Tune parameters in the model such as number of neurons in hidden layers.